In [ ]:
# 📦 IMPORTS AND SETUP (Updated with YOLOv11)
print("🔧 Loading libraries...")

import cv2
import numpy as np
import pandas as pd
import mediapipe as mp
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import json
from pathlib import Path
import time
from typing import List, Dict, Tuple, Optional
from dataclasses import dataclass
import warnings
warnings.filterwarnings('ignore')

# YOLOv11 (Latest version for better performance)
from ultralytics import YOLO

print("✅ Basic libraries loaded")

# Initialize MediaPipe
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

print("✅ MediaPipe initialized")

# Initialize YOLOv11 (Updated from YOLOv8)
print("🔄 Loading YOLOv11 model...")
try:
    # YOLOv11 nano model for speed (will download automatically if not present)
    ball_model = YOLO('yolo11n.pt')  # Changed from yolov8n.pt to yolo11n.pt
    print("✅ YOLOv11 model loaded successfully!")
except Exception as e:
    print(f"⚠️ YOLOv11 loading error: {e}")
    print("🔄 Downloading YOLOv11 model...")
    ball_model = YOLO('yolo11n.pt')
    print("✅ YOLOv11 model downloaded and loaded!")

# Project paths
VIDEO_DIR = Path("video")
OUTPUT_DIR = Path("analysis_results")
OUTPUT_DIR.mkdir(exist_ok=True)

print("✅ Project directories configured")
print("🚀 All libraries loaded - YOLOv11 ready for ball detection!")


In [ ]:
# 🤸‍♂️ POSE DETECTION CLASS
class PoseDetector:
    """
    Advanced pose detection using MediaPipe
    Detects and analyzes human body pose in video frames
    """
    def __init__(self, confidence=0.7):
        self.pose = mp_pose.Pose(
            static_image_mode=False,
            model_complexity=1,
            enable_segmentation=False,
            min_detection_confidence=confidence,
            min_tracking_confidence=confidence
        )
        
    def detect_pose(self, frame):
        """Detect pose landmarks in frame"""
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = self.pose.process(frame_rgb)
        
        if results.pose_landmarks:
            return results.pose_landmarks.landmark, results
        return None, results
    
    def get_key_points(self, landmarks):
        """Extract key body points for analysis"""
        if landmarks is None:
            return {}
        
        # Key points for football analysis
        key_points = {
            'nose': (landmarks[0].x, landmarks[0].y),
            'left_shoulder': (landmarks[11].x, landmarks[11].y),
            'right_shoulder': (landmarks[12].x, landmarks[12].y),
            'left_elbow': (landmarks[13].x, landmarks[13].y),
            'right_elbow': (landmarks[14].x, landmarks[14].y),
            'left_wrist': (landmarks[15].x, landmarks[15].y),
            'right_wrist': (landmarks[16].x, landmarks[16].y),
            'left_hip': (landmarks[23].x, landmarks[23].y),
            'right_hip': (landmarks[24].x, landmarks[24].y),
            'left_knee': (landmarks[25].x, landmarks[25].y),
            'right_knee': (landmarks[26].x, landmarks[26].y),
            'left_ankle': (landmarks[27].x, landmarks[27].y),
            'right_ankle': (landmarks[28].x, landmarks[28].y),
            'left_foot': (landmarks[31].x, landmarks[31].y),
            'right_foot': (landmarks[32].x, landmarks[32].y)
        }
        return key_points

# ⚽ BALL TRACKER CLASS (Updated with YOLOv11)
class BallTracker:
    """
    Advanced ball detection using YOLOv11 (Latest version)
    Faster and more accurate than YOLOv8
    """
    def __init__(self, confidence=0.5):
        self.model = ball_model  # YOLOv11 model initialized earlier
        self.confidence = confidence
        self.ball_class_id = 32  # 'sports ball' class in COCO dataset
        
    def detect_ball(self, frame):
        """Detect football/soccer ball in frame using YOLOv11"""
        # Run YOLOv11 inference (optimized for speed)
        results = self.model(frame, conf=self.confidence, classes=[self.ball_class_id], verbose=False)
        
        detections = []
        for result in results:
            boxes = result.boxes
            if boxes is not None:
                for box in boxes:
                    # Extract bounding box coordinates
                    x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                    confidence = float(box.conf[0].cpu().numpy())
                    
                    # Calculate center and radius
                    center_x = int((x1 + x2) / 2)
                    center_y = int((y1 + y2) / 2)
                    radius = int(max(x2 - x1, y2 - y1) / 2)
                    
                    detection = {
                        'center': (center_x, center_y),
                        'radius': radius,
                        'bbox': (int(x1), int(y1), int(x2), int(y2)),
                        'confidence': confidence
                    }
                    detections.append(detection)
        
        return detections

print("✅ PoseDetector created")
print("✅ BallTracker created with YOLOv11")

# Initialize detectors
pose_detector = PoseDetector()
ball_tracker = BallTracker()

print("🚀 Detection components ready!")


In [ ]:
# 🎬 VIDEO PROCESSOR CLASS (Fixed OpenCV constants)
class VideoProcessor:
    """
    Main video processing pipeline for football analysis
    FIXED VERSION with correct OpenCV constants
    """
    def __init__(self, video_path):
        self.video_path = video_path
        self.cap = cv2.VideoCapture(str(video_path))
        self.fps = self.cap.get(cv2.CAP_PROP_FPS)
        self.frame_count = int(self.cap.get(cv2.CAP_PROP_FRAME_COUNT))
        self.width = int(self.cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        self.height = int(self.cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        self.pose_data, self.ball_data = [], []

        print(f"📹 Video: {video_path.name} | {self.width}x{self.height} | {self.fps:.1f}fps | {self.frame_count/self.fps:.1f}s")

    def visualize_detection(self, frame_number=50):
        """Create visualization of pose and ball detection"""
        # FIXED: Use CAP_PROP_POS_FRAMES (with S) instead of CAP_PROP_POS_FRAME
        self.cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
        ret, frame = self.cap.read()
        
        if not ret:
            print("❌ Could not read frame for visualization")
            return
            
        # Detect pose and ball
        landmarks, pose_results = pose_detector.detect_pose(frame)
        ball_detections = ball_tracker.detect_ball(frame)
        
        # Draw pose
        if pose_results.pose_landmarks:
            mp_drawing.draw_landmarks(
                frame, pose_results.pose_landmarks,
                mp_pose.POSE_CONNECTIONS,
                mp_drawing_styles.get_default_pose_landmarks_style()
            )
        
        # Draw ball detections
        for ball in ball_detections:
            center = ball['center']
            radius = ball['radius']
            confidence = ball['confidence']
            
            cv2.circle(frame, center, radius, (0, 255, 0), 2)
            cv2.putText(frame, f'Ball: {confidence:.2f}', 
                       (center[0]-50, center[1]-radius-10),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
        
        # Save visualization
        viz_path = OUTPUT_DIR / f"detection_sample_{self.video_path.stem}.png"
        cv2.imwrite(str(viz_path), frame)
        print(f"💾 Visualization saved: {viz_path}")

# ⚽ BALL CONTROL ANALYZER CLASS
class BallControlAnalyzer:
    """
    Analyzes ball control patterns and player performance
    """
    def __init__(self):
        self.touch_threshold = 150  # pixels distance for ball-foot contact
        
    def detect_ball_touches(self, pose_data, ball_data):
        """Detect when player touches the ball"""
        touches = []
        
        for pose_frame in pose_data:
            frame_idx = pose_frame['frame_idx']
            key_points = pose_frame['key_points']
            
            # Find ball position at this frame
            ball_at_frame = [b for b in ball_data if b['frame_idx'] == frame_idx]
            
            if ball_at_frame and key_points:
                ball_pos = ball_at_frame[0]['center']
                
                # Check distance to feet
                for foot_name in ['left_foot', 'right_foot']:
                    if foot_name in key_points:
                        foot_pos = key_points[foot_name]
                        foot_pixel = (
                            int(foot_pos[0] * pose_frame['frame_width']),
                            int(foot_pos[1] * pose_frame['frame_height'])
                        )
                        
                        distance = np.sqrt((ball_pos[0] - foot_pixel[0])**2 + 
                                         (ball_pos[1] - foot_pixel[1])**2)
                        
                        if distance < self.touch_threshold:
                            touches.append({
                                'timestamp': pose_frame['timestamp'],
                                'foot': foot_name,
                                'ball_position': ball_pos,
                                'foot_position': foot_pixel,
                                'distance': distance
                            })
        return touches
    
    def analyze_dribbling_pattern(self, ball_trajectory, touches):
        """Analyze dribbling patterns and control"""
        if len(touches) < 2:
            return None
            
        # Calculate metrics
        total_touches = len(touches)
        time_span = touches[-1]['timestamp'] - touches[0]['timestamp']
        touch_frequency = total_touches / max(time_span, 1)
        
        # Foot preference
        left_touches = sum(1 for t in touches if 'left' in t['foot'])
        right_touches = sum(1 for t in touches if 'right' in t['foot'])
        
        foot_preference = {
            'left': left_touches,
            'right': right_touches,
            'preference': 'left' if left_touches > right_touches else 'right' if right_touches > left_touches else 'balanced'
        }
        
        # Control consistency (based on touch intervals)
        touch_intervals = []
        for i in range(1, len(touches)):
            interval = touches[i]['timestamp'] - touches[i-1]['timestamp']
            touch_intervals.append(interval)
        
        consistency_score = 1.0 / (1.0 + np.std(touch_intervals)) if touch_intervals else 0
        
        return {
            'total_touches': total_touches,
            'touch_frequency': touch_frequency,
            'foot_preference': foot_preference,
            'control_consistency': {
                'consistency_score': consistency_score,
                'avg_interval': np.mean(touch_intervals) if touch_intervals else 0
            }
        }
    
    def generate_feedback(self, analysis):
        """Generate coaching feedback based on analysis"""
        feedback = []
        
        if analysis['touch_frequency'] < 1.0:
            feedback.append("Increase ball contact frequency for better control")
        elif analysis['touch_frequency'] > 3.0:
            feedback.append("Good ball contact frequency - maintain this rhythm")
            
        consistency = analysis['control_consistency']['consistency_score']
        if consistency < 0.5:
            feedback.append("Work on consistent touch timing for better control")
        else:
            feedback.append("Excellent touch consistency!")
            
        foot_pref = analysis['foot_preference']
        if foot_pref['preference'] != 'balanced':
            feedback.append(f"Try using both feet more equally (currently {foot_pref['preference']} foot dominant)")
        else:
            feedback.append("Great balanced use of both feet!")
            
        return feedback

print("✅ VideoProcessor created with OpenCV fix")
print("✅ BallControlAnalyzer created")

# Initialize analyzers
ball_control_analyzer = BallControlAnalyzer()

print("🚀 Core analysis system ready!")


In [ ]:
# 🎬 FULL VIDEO ANALYSIS WITH REAL-TIME DISPLAY
print("🎬 Starting Full Football Analysis with Video Display...")
print("=" * 70)

import time
from IPython.display import display, clear_output, HTML
import matplotlib.patches as patches
from matplotlib.animation import FuncAnimation
import cv2

# Analysis Configuration
MAX_FRAMES = 200          # Process up to 200 frames for demo
SKIP_FRAMES = 2          # Process every 2nd frame for speed
RESIZE_WIDTH = 640       # Video display resolution
ANALYSIS_UPDATE_FREQ = 15 # Update analysis every N frames
SAVE_ANNOTATED_VIDEO = True  # Save video with annotations

start_time = time.time()

if VIDEO_DIR.exists() and list(VIDEO_DIR.glob("*.mp4")):
    # Get video for analysis (Change [0] to [1], [2], etc. to select different videos)
    demo_video = list(VIDEO_DIR.glob("*.mp4"))[0]
    print(f"🎬 Analyzing: {demo_video.name} ({demo_video.stat().st_size // 1024 // 1024}MB)")
    
    try:
        # Initialize video processor
        processor = VideoProcessor(demo_video)
        
        # Setup video writer for annotated output
        if SAVE_ANNOTATED_VIDEO:
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            annotated_video_path = OUTPUT_DIR / f"annotated_{demo_video.stem}.mp4"
            video_writer = cv2.VideoWriter(
                str(annotated_video_path), 
                fourcc, 
                processor.fps, 
                (RESIZE_WIDTH, int(processor.height * RESIZE_WIDTH / processor.width))
            )
            print(f"📹 Will save annotated video to: {annotated_video_path}")
        
        # Analysis tracking variables
        frame_analyses = []
        ball_touches = []
        performance_metrics = {
            'total_frames': 0,
            'frames_with_pose': 0,
            'frames_with_ball': 0,
            'ball_touches': 0,
            'avg_ball_confidence': 0,
            'foot_contacts': {'left': 0, 'right': 0}
        }
        
        # Create real-time analysis display
        fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))
        fig.suptitle(f'Real-Time Football Analysis: {demo_video.name}', fontsize=14)
        
        # Initialize plot elements
        ax1.set_title('Current Frame with Detection')
        ax1.axis('off')
        ax2.set_title('Ball Trajectory')
        ax3.set_title('Touch Frequency Over Time')
        ax4.set_title('Performance Metrics')
        
        ball_trajectory_x, ball_trajectory_y = [], []
        touch_times = []
        
        print(f"\n🔄 Processing video frames...")
        print("📊 Real-time analysis will be displayed below")
        
        # Process video frame by frame
        processor.cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
        frame_idx, processed = 0, 0
        last_update_time = time.time()
        
        while processed < MAX_FRAMES:
            # Read frame
            ret, frame = processor.cap.read()
            if not ret:
                break
                
            if frame_idx % SKIP_FRAMES == 0:
                # Resize frame for processing and display
                h, w = frame.shape[:2]
                scale = RESIZE_WIDTH / w
                new_h = int(h * scale)
                frame_resized = cv2.resize(frame, (RESIZE_WIDTH, new_h))
                
                # Create annotated frame for display and saving
                annotated_frame = frame_resized.copy()
                
                # Detect pose
                landmarks, pose_results = pose_detector.detect_pose(frame_resized)
                key_points = pose_detector.get_key_points(landmarks)
                
                # Detect ball
                ball_detections = ball_tracker.detect_ball(frame_resized)
                
                # Draw pose on annotated frame
                if pose_results.pose_landmarks:
                    mp_drawing.draw_landmarks(
                        annotated_frame, pose_results.pose_landmarks,
                        mp_pose.POSE_CONNECTIONS,
                        mp_drawing_styles.get_default_pose_landmarks_style()
                    )
                    performance_metrics['frames_with_pose'] += 1
                
                # Draw ball detections on annotated frame
                current_ball_pos = None
                if ball_detections:
                    performance_metrics['frames_with_ball'] += 1
                    ball_confidences = []
                    
                    for ball in ball_detections:
                        center = ball['center']
                        radius = ball['radius']
                        confidence = ball['confidence']
                        ball_confidences.append(confidence)
                        
                        # Draw ball detection
                        cv2.circle(annotated_frame, center, radius, (0, 255, 0), 3)
                        cv2.circle(annotated_frame, center, 5, (0, 255, 0), -1)  # Center dot
                        cv2.putText(annotated_frame, f'Ball: {confidence:.2f}', 
                                   (center[0]-40, center[1]-radius-10),
                                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                        
                        current_ball_pos = center
                        ball_trajectory_x.append(center[0])
                        ball_trajectory_y.append(center[1])
                    
                    # Update ball confidence average
                    if ball_confidences:
                        performance_metrics['avg_ball_confidence'] = np.mean(ball_confidences)
                
                # Analyze ball touches
                if landmarks and ball_detections and key_points:
                    frame_data = {
                        'frame_idx': frame_idx,
                        'timestamp': frame_idx / processor.fps,
                        'key_points': key_points,
                        'frame_width': RESIZE_WIDTH,
                        'frame_height': new_h
                    }
                    
                    # Check for ball touches
                    for ball in ball_detections:
                        ball_pos = ball['center']
                        
                        for foot_name in ['left_foot', 'right_foot']:
                            if foot_name in key_points:
                                foot_pos = key_points[foot_name]
                                foot_pixel = (
                                    int(foot_pos[0] * RESIZE_WIDTH),
                                    int(foot_pos[1] * new_h)
                                )
                                
                                distance = np.sqrt((ball_pos[0] - foot_pixel[0])**2 + 
                                                 (ball_pos[1] - foot_pixel[1])**2)
                                
                                if distance < 100:  # Touch threshold
                                    performance_metrics['ball_touches'] += 1
                                    performance_metrics['foot_contacts'][foot_name.split('_')[0]] += 1
                                    touch_times.append(frame_idx / processor.fps)
                                    
                                    # Draw touch indicator
                                    cv2.circle(annotated_frame, foot_pixel, 15, (255, 0, 255), 3)
                                    cv2.putText(annotated_frame, 'TOUCH!', 
                                               (foot_pixel[0]-20, foot_pixel[1]-20),
                                               cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 255), 2)
                
                # Add frame info overlay
                timestamp = frame_idx / processor.fps
                info_text = [
                    f"Frame: {frame_idx} | Time: {timestamp:.1f}s",
                    f"Pose: {'✓' if landmarks else '✗'} | Ball: {'✓' if ball_detections else '✗'}",
                    f"Touches: {performance_metrics['ball_touches']}"
                ]
                
                for i, text in enumerate(info_text):
                    cv2.putText(annotated_frame, text, (10, 25 + i*20),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
                    cv2.putText(annotated_frame, text, (10, 25 + i*20),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 1)
                
                # Save annotated frame to video
                if SAVE_ANNOTATED_VIDEO and video_writer:
                    video_writer.write(annotated_frame)
                
                # Update performance metrics
                performance_metrics['total_frames'] = processed + 1
                
                # Update real-time display every N frames
                if processed % ANALYSIS_UPDATE_FREQ == 0 or processed < 5:
                    clear_output(wait=True)
                    
                    # Current frame display
                    ax1.clear()
                    ax1.imshow(cv2.cvtColor(annotated_frame, cv2.COLOR_BGR2RGB))
                    ax1.set_title(f'Frame {frame_idx} - Time: {timestamp:.1f}s')
                    ax1.axis('off')
                    
                    # Ball trajectory
                    ax2.clear()
                    if len(ball_trajectory_x) > 1:
                        ax2.plot(ball_trajectory_x, ball_trajectory_y, 'ro-', alpha=0.6, markersize=3)
                        ax2.set_xlim(0, RESIZE_WIDTH)
                        ax2.set_ylim(0, new_h)
                        ax2.invert_yaxis()
                    ax2.set_title(f'Ball Trajectory ({len(ball_trajectory_x)} points)')
                    ax2.grid(True, alpha=0.3)
                    
                    # Touch frequency
                    ax3.clear()
                    if touch_times:
                        ax3.hist(touch_times, bins=max(1, len(touch_times)//3), alpha=0.7, color='orange')
                    ax3.set_title(f'Ball Touches Over Time ({len(touch_times)} total)')
                    ax3.set_xlabel('Time (seconds)')
                    ax3.set_ylabel('Touch Count')
                    
                    # Performance metrics
                    ax4.clear()
                    metrics_data = [
                        performance_metrics['frames_with_pose'] / max(1, performance_metrics['total_frames']) * 100,
                        performance_metrics['frames_with_ball'] / max(1, performance_metrics['total_frames']) * 100,
                        performance_metrics['avg_ball_confidence'] * 100,
                        min(performance_metrics['ball_touches'] * 10, 100)  # Scale touches
                    ]
                    metrics_labels = ['Pose\nDetection %', 'Ball\nDetection %', 'Ball\nConfidence %', 'Touch\nActivity']
                    colors = ['skyblue', 'lightgreen', 'gold', 'coral']
                    
                    bars = ax4.bar(metrics_labels, metrics_data, color=colors)
                    ax4.set_ylim(0, 100)
                    ax4.set_title('Real-Time Performance Metrics')
                    
                    # Add value labels on bars
                    for bar, value in zip(bars, metrics_data):
                        height = bar.get_height()
                        ax4.text(bar.get_x() + bar.get_width()/2., height + 1,
                                f'{value:.1f}', ha='center', va='bottom', fontsize=9)
                    
                    plt.tight_layout()
                    plt.show()
                    
                    # Print progress
                    elapsed = time.time() - start_time
                    progress = processed / MAX_FRAMES
                    eta = (elapsed / progress - elapsed) if progress > 0 else 0
                    
                    print(f"📊 Progress: {processed}/{MAX_FRAMES} ({progress*100:.1f}%) | "
                          f"Elapsed: {elapsed:.0f}s | ETA: {eta:.0f}s")
                    print(f"⚽ Detections: Pose={performance_metrics['frames_with_pose']}, "
                          f"Ball={performance_metrics['frames_with_ball']}, "
                          f"Touches={performance_metrics['ball_touches']}")
                
                processed += 1
            frame_idx += 1
        
        # Close video writer
        if SAVE_ANNOTATED_VIDEO and video_writer:
            video_writer.release()
            print(f"\n📹 Annotated video saved: {annotated_video_path}")
        
        # Final analysis and report generation
        total_time = time.time() - start_time
        
        print(f"\n🎉 VIDEO ANALYSIS COMPLETE!")
        print(f"⏱️  Total processing time: {total_time//60:.0f}min {total_time%60:.1f}s")
        print(f"📊 Processed {performance_metrics['total_frames']} frames")
        
        # Generate comprehensive report
        report = {
            'video_info': {
                'filename': demo_video.name,
                'total_frames_processed': performance_metrics['total_frames'],
                'processing_time_seconds': total_time,
                'fps': processor.fps
            },
            'detection_performance': {
                'pose_detection_rate': performance_metrics['frames_with_pose'] / performance_metrics['total_frames'],
                'ball_detection_rate': performance_metrics['frames_with_ball'] / performance_metrics['total_frames'],
                'average_ball_confidence': performance_metrics['avg_ball_confidence']
            },
            'ball_control_analysis': {
                'total_ball_touches': performance_metrics['ball_touches'],
                'touch_frequency': performance_metrics['ball_touches'] / (performance_metrics['total_frames'] / processor.fps),
                'foot_preference': performance_metrics['foot_contacts'],
                'ball_trajectory_points': len(ball_trajectory_x)
            }
        }
        
        # Save detailed report
        report_path = OUTPUT_DIR / f"analysis_report_{demo_video.stem}.json"
        with open(report_path, 'w') as f:
            json.dump(report, f, indent=2)
        
        print(f"\n📄 ANALYSIS REPORT:")
        print(f"   🎬 Video: {report['video_info']['filename']}")
        print(f"   📊 Frames processed: {report['video_info']['total_frames_processed']}")
        print(f"   🤸 Pose detection: {report['detection_performance']['pose_detection_rate']*100:.1f}%")
        print(f"   ⚽ Ball detection: {report['detection_performance']['ball_detection_rate']*100:.1f}%")
        print(f"   🏃 Ball touches: {report['ball_control_analysis']['total_ball_touches']}")
        print(f"   🦶 Foot preference: L:{report['ball_control_analysis']['foot_preference']['left']} R:{report['ball_control_analysis']['foot_preference']['right']}")
        
        # Generate coaching feedback
        feedback = []
        
        if report['detection_performance']['pose_detection_rate'] > 0.8:
            feedback.append("✅ Excellent pose visibility throughout the video")
        elif report['detection_performance']['pose_detection_rate'] > 0.5:
            feedback.append("⚠️ Good pose detection - consider better camera angle for improvement")
        else:
            feedback.append("❌ Poor pose detection - check camera position and lighting")
            
        if report['ball_control_analysis']['total_ball_touches'] > 5:
            feedback.append("⚽ Good ball control activity detected")
        else:
            feedback.append("⚽ Limited ball contact detected - focus on ball control drills")
            
        left_touches = report['ball_control_analysis']['foot_preference']['left']
        right_touches = report['ball_control_analysis']['foot_preference']['right']
        if abs(left_touches - right_touches) > 3:
            dominant_foot = 'left' if left_touches > right_touches else 'right'
            feedback.append(f"🦶 {dominant_foot.capitalize()} foot dominant - practice with both feet")
        else:
            feedback.append("🦶 Good balanced use of both feet")
        
        print(f"\n💬 COACHING FEEDBACK:")
        for i, fb in enumerate(feedback, 1):
            print(f"   {i}. {fb}")
        
        print(f"\n📁 FILES GENERATED:")
        if SAVE_ANNOTATED_VIDEO:
            print(f"   🎬 Annotated video: {annotated_video_path}")
        print(f"   📄 Analysis report: {report_path}")
        print(f"   📊 Performance charts: Displayed above")
        
        print(f"\n🏆 FOOTBALL ANALYSIS SYSTEM COMPLETE!")
        
    except Exception as e:
        print(f"❌ Error during analysis: {e}")
        import traceback
        traceback.print_exc()
        
else:
    print("❌ No video files found in 'video' directory!")
    print("📁 Please add .mp4 files to analyze")
